## <div style="color: #E98E29">**1. Ler PDF completo** </div>

 - <s style="color: #92BB2D">CPFL</s>
 - COPEL
 - CEMIG
 - <s style="color: #92BB2D">EDP</s>
 - ELEKTRO
 - ENEL

---

## <div style="color: #E98E29">**2. Criar a base**</div>
- <s style="color: #92BB2D">Coletar Meses e ano(s)</s>
- <s style="color: #92BB2D">Coletar Consumos</s>
-  <s style="color: #92BB2D">Coletar Demandas</s>
- <s style="color: #92BB2D">Criar DataFrame</s>
- Adicionar a base de dados
<br>
<br>
---

## <s style="color: #92BB2D">**3. Não adicionar datas repetidas**</s>

<br>

## LER OS PDFS

In [1]:
import sys

classes = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\classes"
funcs = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\funcoes"
sys.path.append(classes)
sys.path.append(funcs)

In [2]:
import os
import contextlib
import regex as re
import pandas as pd
from funcoes.f_database import *
from typing import Any, List, Tuple
from Fatura import Fatura

join_path = lambda y, x: os.path.join(y, x)
filename = lambda path, filetype: path.split("\\")[-1].split(filetype)[0]

PATH = r"Q:\APOIO ATENDIMENTO\Faturas EcoExp\Setembro"
FOLDERS = os.listdir(PATH)
PATHS = [{filename(x, ".pdf"): join_path(PATH, x)} for x in FOLDERS]

# SCRIPTS

## ADICIONAR IDS

In [ ]:
# ADICIONA NOVOS CLIENTES
ucs_df = df[["nome"]].drop_duplicates().reset_index(drop=True)
db_path = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\banco_de_dados\distribuidoras.db"
ucs_db = get_clientes_existentes(db_path)
ucs_nova = adiciona_novo_id_clientes(mantem_novos_clientes(ucs_df, ucs_db))
if not (ucs_nova.empty):
    print(f"Clientes a adicionar: {', '.join(ucs_nova.nome.values.tolist())}")
    if salva_novos_clientes(ucs_nova, db_path):
        print("Insercao de Novos Clientes ok")
        ucs_db = get_clientes_existentes(db_path)
    else:
        print("Erro ao salvar valores no banco")
else:
    print("Nenhuma UC nova")

consumos_pdf = criar_id_consumos(df_id=ucs_db, df_consumo=df.drop_duplicates(subset=["nome", "datas"]))
consumos_db = get_consumos_existentes(db_path)
novos_consumos = mantem_novos_consumos(
    novos_consumos=consumos_pdf, consumos_existentes=consumos_db
)
if not(novos_consumos.empty):
    status = salva_novos_consumos(novos_consumos=novos_consumos, db_path=db_path)

## DATAFRAME FINAL

In [3]:
from Identificador import Identificador
DISTRIBUIDORAS = {}
for info in PATHS:
    with contextlib.suppress(IndexError,TypeError):
        path = info.get(*info.keys())
        DISTRIBUIDORA = Identificador(infos=info).main()(path)
        name = DISTRIBUIDORA.distribuidora
        if name not in DISTRIBUIDORAS.keys():
            DISTRIBUIDORAS[name] = []
        if name != None:
            DISTRIBUIDORAS[name].append(DISTRIBUIDORA)


# NOVAS CLASSES - DESENVOLVIMENTO

In [42]:
class COPEL(Fatura):
    def __init__(self, path: str):
        super().__init__(path)
    
    def _consumo_posicao(self, energia_descricao: str) -> int:
        """
        Encontra o índice de início da descrição de energia na página da fatura.

        Args:
            energia_descricao (str): A descrição da energia a ser encontrada (consumo ponta ou fora ponta).

        Returns:
            int: O índice de início da descrição de energia.
        """

        indice = self._ultima_pagina.find(energia_descricao)
        assert (
            indice != -1
        ), f"Tipo de energia não encontrado, esperado: 'consumo ponta' ou 'consumo fora de ponta' || Obtido {energia_descricao} - Cliente {self._nome}"
        self.indice = indice
        return indice

    def _encontra_ano_anterior(self, datas: str):
        """
        Retorna o ano A-1 com base nas datas fornecidas.

        Args:
            datas (str): Uma string contendo datas no formato 'Mês/Ano'.

        Returns:
            int: O ano A-1.
        """
        return int(datas[:4]) - 1

    def _find_consumo(self, energia_descricao: int) -> List[str]:
        """
        Extrai informações de consumo da página da fatura.

        Args:
            indice (int): O índice de início da busca na página.

        Returns:
            List[str]: Uma lista de strings contendo informações de consumo.
        """
        indice = self._consumo_posicao(energia_descricao)
        self.medida_consumo = indice
        lista_consumo = self._lista_consumo(indice)
        energia_ind, dias_ind = self._get_indices(lista_consumo)
        return lista_consumo[energia_ind+1: dias_ind]

    def _get_indices(self, lista: List[str]) -> Tuple[int, int]:
        return lista.index("energia_wh"), lista.index("dias")

    def _lista_consumo(self, indice) -> List[str]:
        return (
            self._ultima_pagina[indice:]
            .replace("kwh", "\nenergia_wh")
            .replace("mwh", "\nenergia_wh")
            .replace(" ", "")
            .replace("dias", "\ndias")
            .split("\n")
        )
    
    @Fatura.distribuidora.getter
    def distribuidora(self) -> str:
        return self.__class__.__name__

    @Fatura.consumo.setter
    def consumo(self, flag: Any):       
        self._consumo = "fazendo"
    
    @Fatura.data.setter
    def data(self, primeiro_indice: int):
        if not self._data:
            self._data = "fazendo"

    @Fatura.medida_consumo.setter
    def medida_consumo(self, flag: Any) -> None:
        self._medida_consumo = re.search("consumo \((\wwh)", self.ultima_pagina).group(1)

    @Fatura.medida_demanda.setter
    def medida_demanda(self, flag: Any) -> None:
        self._medida_demanda = re.search("demanda \((\ww)", self.ultima_pagina).group(1)

    @Fatura.nome.setter
    def nome(self, flag: Any):
        if self._ultima_pagina.split("\n")[1] != "segunda via":
            self._nome = self._ultima_pagina.split("\n")[1]
        else:
            self._nome = self._ultima_pagina.split("\n")[3]
           
    def main(self):
        self.nome = None
        self.medida_consumo = None
        self.medida_demanda = None

caminhos = [copel._caminho for copel in DISTRIBUIDORAS['COPEL']]

In [14]:
def nome(copel):
    if copel._ultima_pagina.split("\n")[1] != "segunda via":
        copel._nome = copel._ultima_pagina.split("\n")[1]
    else:
        copel._nome = copel._ultima_pagina.split("\n")[3]


In [128]:
for caminho in caminhos:
    print(caminho)
    copel = COPEL(caminho)
    copel.main()
    cols = {0: "mes", 1:"valor", 2:"data_venc", 3: "data_pgto", 4: "consumo_ponta", 5:"consumo_fora_de_ponta", 6: "demanda_ponta", 7: "demanda_fora_de_ponta"}
    index = copel.ultima_pagina.find('mês/ano faturavencimento pagamento pontafora pta. pontafora pta. pontafora pta. pontafora pta.')
    c = copel.ultima_pagina[index:].split("\n")[1:]
    c = list(map(lambda x: x.split(" "), c))
    df = pd.DataFrame(
        list(
            map(
                lambda i:list(filter(lambda x: x!="", i)),
                c
                )
            )
        )
    df = df[df.columns[:8]].rename(columns=cols)
    break

Q:\APOIO ATENDIMENTO\Faturas EcoExp\Setembro\100215513.pdf


In [129]:
def arruma_data_sempgto(df: pd.DataFrame) -> pd.DataFrame:
    """
    Algumas datas de pgto mais recentes ainda nao processaram ou nao receberam pgto e estao em branco, por isso, 
    os dados de consumo podem ser deslocados para a posicao dessa data, desarrumando o dataframe, o script checa se o valor é uma data
    caso nao seja, faz as alteracoes nas demais colunas e no valor da coluna de data_pgto
    """
    if df["data_pgto"][0].find("/") == -1:
        colunas_incorretas = ["data_pgto", "consumo_ponta", "consumo_fora_de_ponta", "demanda_ponta"]
        colunas_corretas = ["consumo_ponta", "consumo_fora_de_ponta", "demanda_ponta", "demanda_fora_de_ponta"]
        df = atribui_valores_corretos(df, colunas_corretas, colunas_incorretas)
    return df


def arruma_consumos_nao_separados(df: pd.DataFrame) -> pd.DataFrame:
    """
    Alguns pdf's são lidos de forma incorreta, não separando os consumos P e FP (100kWh P e 1000kWh FP podem ser lidos como 1001000)
    o script separa esses consumos e os ajusta suas respectivas colunas
    """
    if checa_diferenca(df, 100):
        num_chars = len(df["consumo_ponta"].iloc[1])
        df, p, fp = atribui_demanda_P_e_FP(df, num_chars)
        df.loc[0].consumo_ponta = p
        df.loc[0].consumo_fora_de_ponta = fp
    return df


def atribui_demanda_P_e_FP(df: pd.DataFrame, num_chars: int) -> Tuple[pd.DataFrame, str, str]:
        p, fp = df.consumo_ponta.iloc[0][:num_chars], df.consumo_ponta.iloc[0][num_chars:]
        df.loc[0, ["demanda_ponta", "demanda_fora_de_ponta"]] = df[["consumo_fora_de_ponta", "demanda_ponta"]].iloc[0].values
        return df, p, fp


def atribui_valores_corretos(df: pd.DataFrame, colunas_corretas: List[str], colunas_incorretas: List[str]) -> pd.DataFrame:
    df.loc[0, colunas_corretas] = df[colunas_incorretas].iloc[0].values
    df.loc[0, ["data_pgto"]] = "01/01/1900"
    return df


def checa_diferenca(df, lim):
    return df["consumo_ponta"].astype(float).iloc[0] / df["consumo_ponta"].astype(float).iloc[1] > lim


,mes,valor,data_venc,data_pgto,consumo_ponta,consumo_fora_de_ponta,demanda_ponta,demanda_fora_de_ponta
0,10/2023,"51.706,03",05/11/2023,01/01/1900,585,104672,24.19,335.23
1,09/2023,"27.062,99",05/10/2023,05/10/2023,596,48573,57.02,258.33
2,08/2023,"9.628,14",05/09/2023,05/09/2023,431,12070,9.5166.75,0
3,07/2023,"3.919,94",05/08/2023,07/08/2023,363,5503,8.64,65.66
4,06/2023,"4.057,38",05/07/2023,05/07/2023,324,6470,7.77,65.66
5,05/2023,"7.933,95",05/06/2023,05/06/2023,306,14661,7.77,116.64
6,04/2023,"26.457,92",05/05/2023,05/05/2023,432,56882,8.64,254.88
7,03/2023,"7.022,31",05/04/2023,05/04/2023,254,7817,6.91,204.76
8,02/2023,"2.214,05",05/03/2023,06/03/2023,287,3176,6.91,17.28
9,01/2023,"4.668,18",05/02/2023,06/02/2023,377,5964,67.39,104.54
